In [50]:
#Install packages
#!pip install python-twitter
!pip install TwitterAPI

In [23]:
# Import packages
import json
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from TwitterAPI import TwitterAPI
from nltk import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from string import punctuation
from bs4 import BeautifulSoup
#nltk.download('punkt')
#nltk.download('stopwords')
from nltk.corpus import stopwords 

In [434]:
#data_file = "/Users/nescobar/Dropbox/Indiana/Social_Media_Mining/Project/smm2018/data/raw/tweets_harvey_20180218.csv"
train_file = "/Users/nescobar/Dropbox/Indiana/Social_Media_Mining/Project/smm2018/data/raw/csv/labelled_edited.csv"
test_file = "/Users/nescobar/Dropbox/Indiana/Social_Media_Mining/Project/smm2018/data/raw/tweets_harvey_recent.csv"

consumer_key = 'FP94BfUftKeygulmcLcVqRNvX'
consumer_secret = 'GfZtNuk6Zu6lTOYGWibXI95MjMyks6SSlBEykyLuYe4NEUgGUu'
access_token_key = '961088185756393472-FopLzpw7n3CrHhbHoWv8BlnR1mZwhGH'
access_token_secret= '62Trp0LVEPvAwoGPH4ov8D4TQe2eEaQETKIfXKCPZX6NN'

In [249]:
api = TwitterAPI(consumer_key, consumer_secret, auth_type='oAuth2')

In [451]:
class ProcessTweets:
    
    def __init__(self):
        self._stopwords=set(stopwords.words('english')+list(punctuation)+['ATUSER','URL','IMG'])
        
    def get_data(query="", source="file", path=train_file, feed="search/tweets",api=api, maxid=0, n=100):
        try:
            if source == "file":
                harvey_df = pd.read_csv(path, encoding = 'ISO-8859-1')
                return harvey_df
            else:
                if maxid == 0:
                    return pd.read_json(json.dumps([t for t in api.request(feed, {'q':query,'count':n})]))
                else:
                    return pd.read_json(json.dumps([t for t in api.request(feed, {'q': query, 'count': n, 'max_id': maxid})]))            
        except:
            print("Error while getting data")
            return None
    
    def process_tweets(self, list_of_tweets):
        processed_tweets=[]
        for tweet in list_of_tweets:
            processed_tweets.append((self._process_tweet(tweet)))
        return processed_tweets
    
    def _process_tweet(self,tweet):
        try:
            # Unescape from HTML
            #tweet = html.unescape(tweet)
            tweet = BeautifulSoup(tweet, 'lxml').get_text()
            # 3a. Convert to lower case
            tweet = tweet.lower()
            # 3b. Replace links with the word URL 
            tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',tweet) 
            # 3c. Replace @username with "AT_USER"
            tweet = re.sub('@[^\s]+',' ',tweet)                
            # 3d. Replace #word with word 
            tweet = re.sub(r'#([^\s]+)',r'\1',tweet)
            # 3e. Replace images with the word IMG 
            tweet = re.sub(r'\bpic.twitter.com\s+', ' ', tweet)
            # 3f Keep only words with letters
            tweet = re.sub('[^a-zA-Z]',' ',tweet)
            # 3g. Remove RT
            tweet = re.sub(r'\brt([\b\s])', ' ', tweet)
            # Apply Lemmatization
            lemmatizer = WordNetLemmatizer()           
            tweet = [lemmatizer.lemmatize(word) for word in tweet.split() if word not in self._stopwords and len(word)>1]  
            return (" ".join(tweet)).strip()
        except:
            print("Error with tweet: ", tweet)
            return None

In [452]:
# Process tweets from file
tweet_processor = ProcessTweets()
raw_tweets = tweet_processor.get_data()
#[print(t) for t in raw_tweets['text'][:10]]
#cleaned_tweets = tweet_processor.process_tweets(raw_tweets['text'])

In [341]:
# Create train DF 
train_df = pd.DataFrame(cleaned_tweets, columns=['text'])
train_df['target'] = raw_tweets.label
train_df=train_df.drop_duplicates('text')
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 387 entries, 0 to 406
Data columns (total 2 columns):
text      387 non-null object
target    387 non-null object
dtypes: object(2)
memory usage: 9.1+ KB


In [463]:
# Top ngrams in tweets
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

filter_file = "/Users/nescobar/Dropbox/Indiana/Social_Media_Mining/Project/data/csv/filter_tweets_transportation.csv"

filtered_tweets = tweet_processor.get_data(path=filter_file)
cleaned_filtered_tweets = tweet_processor.process_tweets(filtered_tweets['text'])

vect = TfidfVectorizer(ngram_range=(2,5))
summaries = "".join(cleaned_filtered_tweets)
ngrams_summaries = vect.build_analyzer()(summaries)

[bigram for bigram in Counter(ngrams_summaries).most_common(20) if "harvey" not in bigram[0]]

[('public transportation', 12),
 ('transportation network', 9),
 ('need transportation', 8),
 ('transportation infrastructure', 7),
 ('transportation shelter', 5),
 ('state transportation', 5),
 ('transportation houston', 5),
 ('evacuee back', 5),
 ('people transportation', 4),
 ('without transportation', 4),
 ('help transportation', 4),
 ('transportation system', 4),
 ('dog houston', 4)]

In [365]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, f1_score, recall_score, accuracy_score

# Train/test split training data
X_train, X_test, y_train, y_test = train_test_split(train_df.text, train_df.target, test_size=0.5, random_state=42)

# Build a dictionary of feature indices
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
print(X_train_vec.shape)

# Apply TF-IDF to the dictionary
tfidf = TfidfTransformer()
X_train_tfidf = tfidf.fit_transform(X_train_vec)
print(X_train_tfidf.shape)

(193, 799)
(193, 799)


In [366]:
# Testing with Naive Bayes
from sklearn.naive_bayes import MultinomialNB
from time import time
from sklearn.pipeline import Pipeline
from sklearn import metrics

# Start running time
t0 = time()

# Build a pipeline that contains vectorizer, transform and classifier
naive_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
                     ])

naive_clf.fit(X_train, y_train)

# Get test data from recent tweets
#test_df = tweet_processor.get_data(dtype="test")

# Clean up testing data set
clean_test_df = tweet_processor.process_tweets(y_test)

# Test the classifier
predicted = naive_clf.predict(clean_test_df)

# Print running time for training and predicting
print('Total running time', time() - t0)

# Evaluate accuracy of the model
accuracy = accuracy_score(y_test, predicted)
print(accuracy)

print(metrics.classification_report(y_test, predicted))
# Print predicted category for each tweet
#for tweet, category in zip(test_tweets.text, predicted):
#    print('%r => %s' % (tweet, category))
    

Total running time 0.20992588996887207
0.896907216495
                precision    recall  f1-score   support

       Animals       1.00      1.00      1.00        22
      Donation       0.71      1.00      0.83        50
         Money       0.00      0.00      0.00        20
    Relocation       1.00      1.00      1.00        47
Transportation       1.00      1.00      1.00         9
  Volunteering       1.00      1.00      1.00        46

   avg / total       0.82      0.90      0.85       194



/Users/nescobar/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [367]:
# SVM
from sklearn.linear_model import SGDClassifier
from time import time
from sklearn.pipeline import Pipeline
from sklearn import metrics

# Start running time
t0 = time()

# Build a pipeline that contains vectorizer, transform and classifier
svm_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, random_state=42,
                                            max_iter=5, tol=None)),
                     ])

svm_clf.fit(X_train, y_train)

# Clean up testing data set
clean_test_df = tweet_processor.process_tweets(y_test)

# Test the classifier
predicted = svm_clf.predict(clean_test_df)

# Print running time for training and predicting
print('Total running time', time() - t0)

# Evaluate accuracy of the model
accuracy = accuracy_score(y_test, predicted)
print(accuracy)

print(metrics.classification_report(y_test, predicted))

Total running time 0.24737000465393066
1.0
                precision    recall  f1-score   support

       Animals       1.00      1.00      1.00        22
      Donation       1.00      1.00      1.00        50
         Money       1.00      1.00      1.00        20
    Relocation       1.00      1.00      1.00        47
Transportation       1.00      1.00      1.00         9
  Volunteering       1.00      1.00      1.00        46

   avg / total       1.00      1.00      1.00       194



In [ ]:
# Visualization